helper functions

In [ ]:
import matplotlib.image as mpimg
import numpy as np
import cv2
from skimage.feature import hog
import matplotlib.pyplot as plt

def convert_color(img, conv='RGB2YCrCb'):
    if conv == 'RGB2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    if conv == 'BGR2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    if conv == 'RGB2LUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
    if conv == 'RGB2HLS':
        return cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    if conv == 'RGB2HSV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    if conv == 'RGB2RGB':
        return img


# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)

        return features

# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

# Define a function to compute color histogram features 
# NEED TO CHANGE bins_range if reading .png files with mpimg!
def color_hist(img, nbins=32):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features
    
# Define a function that takes an image,
# start and stop positions in both x and y, 
# window size (x and y dimensions),  
# and overlap fraction (for both x and y)
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

# Define a function to extract features from a single image window
# This function is very similar to extract_features()
# just for a single image rather than list of images
def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)

    #9) Return concatenated array of features
    return np.concatenate(img_features)

# Define a function you will pass an image 
# and the list of windows to be searched (output of slide_windows())
def search_windows(img, windows, clf, scaler, color_space='RGB', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=True, hog_feat=True):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
        #4) Extract features for that window using single_img_features()
        features = single_img_features(test_img, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows

def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap

def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)

    # Return the image
    return img

# Define a single function that can extract features using hog sub-sampling and make predictions
def find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, color_space):
    
    #draw_img = np.copy(img)
    img = img.astype(np.float32)/255
    
    img_tosearch = img[ystart:ystop,:,:]
    conv = 'RGB2' + color_space
    ctrans_tosearch = convert_color(img_tosearch, conv=conv)
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
        
    ch1 = ctrans_tosearch[:,:,0]
    ch2 = ctrans_tosearch[:,:,1]
    ch3 = ctrans_tosearch[:,:,2]

    # Define blocks and steps as above
    nxblocks = (ch1.shape[1] // pix_per_cell) - cell_per_block + 1
    nyblocks = (ch1.shape[0] // pix_per_cell) - cell_per_block + 1 
    nfeat_per_block = orient*cell_per_block**2
    
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step
    
    # Compute individual channel HOG features for the entire image
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
    
    bbox_list = []

    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            # Extract HOG for this patch
            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell

            # Extract the image patch
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))
          
            # Get color features
            spatial_features = bin_spatial(subimg, size=spatial_size)
            hist_features = color_hist(subimg, nbins=hist_bins)

            # Scale features and make a prediction
            test_features = X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1))    
            #test_features = X_scaler.transform(np.hstack((shape_feat, hist_feat)).reshape(1, -1))    
            test_prediction = svc.predict(test_features)
            
            if test_prediction == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)

                #cv2.rectangle(draw_img,(xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart),(0,0,255),6) 
                
                box = ((xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart))
                #print(box)
                bbox_list.append(box)
                #print(bbox_list)
    #((x1, y1), (x2, y2))            
    #return draw_img

    return bbox_list

Image Pipeline

In [4]:
import matplotlib.image as mpimg
import numpy as np
import cv2
import glob
import pickle
import glob
import time
import collections

from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.model_selection import train_test_split

from scipy.ndimage.measurements import label
from moviepy.editor import VideoFileClip

class Pipeline:

    def __init__(self):
        print('Pipeline object created ...')
        self.frame = 1
        self.heatmaps = collections.deque(maxlen=30) 
        self.pickle_cache = {}
        
    def get_image_files(self):
        fn_cars = glob.glob('./data/vehicles/*/*.png')
        fn_notcars = glob.glob('./data/non-vehicles/*/*.png')

        print('Number of car images:', len(fn_cars))
        print('Number of non-car images:', len(fn_notcars))

        return {'cars':fn_cars, 'notcars':fn_notcars}


    def train_classifier(self):

        fns = self.get_image_files()
        cars = fns['cars']
        notcars = fns['notcars']

        ### TODO: Tweak these parameters and see how the results change.
        colorspaces = ['RGB', 'HSV',  'HLS', 'YCrCb']
        #color_space = 'HLS' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
        orient = 9  # HOG orientations
        pix_per_cell = 8 # HOG pixels per cell
        cell_per_block = 2 # HOG cells per block
        hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
        spatial_size = (16, 16) # Spatial binning dimensions
        hist_bins = 16    # Number of histogram bins
        spatial_feat = True # Spatial features on or off
        hist_feat = True # Histogram features on or off
        hog_feat = True # HOG features on or off
        y_start_stop = [None, None] # Min and max in y to search in slide_window()

        for color_space in colorspaces:
            print('Extracting features for color_space', color_space)
            car_features = extract_features(cars, color_space=color_space, 
                                        spatial_size=spatial_size, hist_bins=hist_bins, 
                                        orient=orient, pix_per_cell=pix_per_cell, 
                                        cell_per_block=cell_per_block, 
                                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                                        hist_feat=hist_feat, hog_feat=hog_feat)
            notcar_features = extract_features(notcars, color_space=color_space, 
                                        spatial_size=spatial_size, hist_bins=hist_bins, 
                                        orient=orient, pix_per_cell=pix_per_cell, 
                                        cell_per_block=cell_per_block, 
                                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                                        hist_feat=hist_feat, hog_feat=hog_feat)
            X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
            # Fit a per-column scaler
            X_scaler = StandardScaler().fit(X)
            # Apply the scaler to X
            scaled_X = X_scaler.transform(X)

            # Define the labels vector
            y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))


            # Split up data into randomized training and test sets
            rand_state = np.random.randint(0, 100)
            X_train, X_test, y_train, y_test = train_test_split(
                    scaled_X, y, test_size=0.2, random_state=rand_state)

            print('Using:',orient,'orientations',pix_per_cell,
                    'pixels per cell and', cell_per_block,'cells per block')
            print('Feature vector length:', len(X_train[0]))

            # Use a linear SVC 
            svc = LinearSVC()
            # Check the training time for the SVC
            t=time.time()
            svc.fit(X_train, y_train)
            t2 = time.time()

            print(round(t2-t, 2), 'Seconds to train SVC...')
            # Check the score of the SVC
            test_accuracy = round(svc.score(X_test, y_test), 4)
            print('Test Accuracy of SVC = ', test_accuracy)
            # Check the prediction time for a single sample
            t=time.time()

            dist_pickle = {}
            dist_pickle["svc"] = svc
            dist_pickle["scaler"] = X_scaler
            dist_pickle["orient"] = orient
            dist_pickle["pix_per_cell"] = pix_per_cell
            dist_pickle["cell_per_block"] = cell_per_block
            dist_pickle["spatial_size"] = spatial_size
            dist_pickle["hist_bins"] = hist_bins
            dist_pickle["test_accuracy"] = test_accuracy
            dist_pickle["color_space"] = color_space

            pickle_file = 'svc_pickle_' + color_space + '.p'
            pickle.dump( dist_pickle, open(pickle_file, 'wb') )

    def load_pickle(self, pickle_file):
        if (pickle_file not in self.pickle_cache):
            dist_pickle = pickle.load( open(pickle_file, "rb" ) )
            self.pickle_cache[pickle_file] = dist_pickle
        else:
            dist_pickle = self.pickle_cache[pickle_file]

        return dist_pickle

    def detect_cars(self, img):
        ystart = 400
        ystop = 656
        scales = [1.0, 1.5, 1.75, 2.0]

        color_space = 'YCrCb' # YRcCB had the highest test accuracy

        dist_pickle = self.load_pickle("svc_pickle_" + color_space + ".p")
        svc = dist_pickle["svc"]
        X_scaler = dist_pickle["scaler"]
        orient = dist_pickle["orient"]
        pix_per_cell = dist_pickle["pix_per_cell"]
        cell_per_block = dist_pickle["cell_per_block"]
        spatial_size = dist_pickle["spatial_size"]
        hist_bins = dist_pickle["hist_bins"]
        test_accuracy = dist_pickle["test_accuracy"]
        color_space = dist_pickle["color_space"]

        bbox_list = []
        for scale in scales:
            boxes = find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, color_space)
            bbox_list.extend(boxes)

        return bbox_list

    def process_image(self, image):
        heat = np.zeros_like(image[:,:,0]).astype(np.float)

        # get bounding boxes for all detections
        bbox_list = pipeline.detect_cars(image)

        # Add heat to each box in box list
        heat = add_heat(heat, bbox_list)
    
        # Apply threshold to help remove false positives
        heat = apply_threshold(heat, 1)

        self.heatmaps.append(heat)
        
        
        if len(self.heatmaps) == 30:   
            heat = sum(list(self.heatmaps))
            heat = apply_threshold(heat, 5)
        
        # Visualize the heatmap when displaying    
        heatmap = np.clip(heat, 0, 255)
        
        # Find final boxes from heatmap using label function
        labels = label(heatmap)
        draw_img = draw_labeled_bboxes(np.copy(image), labels)

        # uncomment for visualization during render
        '''
        vis_image = np.copy(image)
        for box in bbox_list:
            cv2.rectangle(vis_image,box[0],box[1],(0,0,255),6)
        fig = plt.figure(figsize=(9,3))
        plt.subplot(121)
        plt.imshow(vis_image)
        plt.title('Car Positions')
        plt.subplot(122)
        plt.imshow(heatmap, cmap='hot')
        plt.title('Heat Map')
        fig.tight_layout()
        fig.savefig('./output_images/vis-frame-' + str(self.frame) + '.png')

        fig = plt.figure()
        plt.imshow(draw_img)
        plt.title('Detected Cars')
        fig.tight_layout()
        fig.savefig('./output_images/vis-frame-' + str(self.frame) + '-detected.png')
        fig = plt.figure()
        plt.imshow(labels[0], cmap='gray')
        plt.title('Labels - ' + str(labels[1]) + ' cars found')
        fig.tight_layout()
        fig.savefig('./output_images/vis-frame-' + str(self.frame) + '-labels.png')
        '''

        self.frame = self.frame + 1


        return draw_img


    def create_output_video(self):
        
        #project_video
#         input_video_file = './project_video.mp4'
#         output_video_file = './project_video_output_2.mp4'
        
        input_video_file = './test_video.mp4'
        output_video_file = './test_video_output3.mp4'

#         start = 15
#         end = 31
#         clip1 = VideoFileClip(input_video_file).subclip(start, end)
        clip1 = VideoFileClip(input_video_file)
        proj_clip = clip1.fl_image(self.process_image) #NOTE: this function expects color images!!

        proj_clip.write_videofile(output_video_file , audio=False)
        
if __name__ == "__main__":
    pipeline = Pipeline()
#     pipeline.train_classifier()


Pipeline object created ...


In [5]:
pipeline = Pipeline()

pipeline.create_output_video()


Pipeline object created ...
[MoviePy] >>>> Building video ./test_video_output3.mp4
[MoviePy] Writing video ./test_video_output3.mp4


 97%|█████████▋| 38/39 [01:00<00:01,  1.63s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./test_video_output3.mp4 



In [ ]:
import matplotlib.pyplot as plt
from random import randint

class Visualize:
    def __init__(self):
        print('Visualize object created ...')
        self.pipeline = Pipeline()

        self.cars = []
        self.notcars = []

        fns = self.pipeline.get_image_files()
        self.cars = fns['cars']
        self.notcars = fns['notcars']
        
    def visualize_input_images(self):
        p = self.pipeline

        rc = randint(0, len(self.cars))
        print('Car', rc)

        rnc = randint(0, len(self.notcars))
        print('Not car', rnc)

        print('Example of a car image:', self.cars[rc])
        print('Example of a non-car image:', self.notcars[rnc])	

        car_img = mpimg.imread(self.cars[rc])
        notcar_img = mpimg.imread(self.notcars[rnc])

        fig = plt.figure()
        plt.subplot(121)
        plt.imshow(car_img)
        plt.title('Car')
        plt.subplot(122)
        plt.imshow(notcar_img)
        plt.title('Not car')
        fig.tight_layout()
        plt.show()
        fig.savefig('./output_images/vis-car_not_car.png')
        
    def visualize_hog(self):
        rc = randint(0, len(self.cars))
        print('Car', rc)

        rnc = randint(0, len(self.notcars))
        print('Not car', rnc)
        
        car_img = mpimg.imread(self.cars[rc])
        notcar_img = mpimg.imread(self.notcars[rnc])

        colorspaces = ['RGB', 'HSV', 'LUV', 'HLS', 'YCrCb']
        for color_space in colorspaces:
            print('Extracting hog -', color_space)
            self.display_hog(car_img, color_space, 'Car')
            self.display_hog(notcar_img, color_space, 'Not-Car')
    
    def display_hog(self, input_img, color_space, title):
        orient = 9  # HOG orientations
        pix_per_cell = 8 # HOG pixels per cell
        cell_per_block = 2 # HOG cells per block
        scale = 1
        img_tosearch = input_img.astype(np.float32)/255

        conv = 'RGB2' + color_space
        ctrans_tosearch = convert_color(img_tosearch, conv=conv)
        if scale != 1:
            imshape = ctrans_tosearch.shape
            ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
    
        ch1 = ctrans_tosearch[:,:,0]
        ch2 = ctrans_tosearch[:,:,1]
        ch3 = ctrans_tosearch[:,:,2]

        # Define blocks and steps as above
        nxblocks = (ch1.shape[1] // pix_per_cell) - cell_per_block + 1
        nyblocks = (ch1.shape[0] // pix_per_cell) - cell_per_block + 1 
        nfeat_per_block = orient*cell_per_block**2

        # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
        window = 64
        nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1
        cells_per_step = 2  # Instead of overlap, define how many cells to step
        nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
        nysteps = (nyblocks - nblocks_per_window) // cells_per_step

        # Compute individual channel HOG features for the entire image
        f1, hog_img1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, vis=True, feature_vec=False)
        f2, hog_img2  = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, vis=True, feature_vec=False)
        f3, hog_img3  = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, vis=True, feature_vec=False)

        fig = plt.figure()
        fig.set_figheight(5)
        fig.set_figwidth(20)
        plt.subplot(141)
        plt.imshow(input_img)
        plt.title(title)
        plt.subplot(142)
        plt.imshow(hog_img1)
        plt.title('Ch1 - ' + color_space)
        plt.subplot(143)
        plt.imshow(hog_img2)
        plt.title('Ch2 - ' + color_space)
        plt.subplot(144)
        plt.imshow(hog_img3)
        plt.title('Ch3 - ' + color_space)

        fig.tight_layout()
        plt.show()
        fn = './output_images/vis-hog-' + color_space + '-' + title + '.png'
        print(fn)
        fig.savefig(fn)
    
    def visualize_search_windows(self, image_file):
        image = mpimg.imread(image_file)
        draw_image = np.copy(image)

        dist_pickle = self.pipeline.load_pickle("svc_pickle_YCrCb.p")
        svc = dist_pickle["svc"]
        X_scaler = dist_pickle["scaler"]
        orient = dist_pickle["orient"]
        pix_per_cell = dist_pickle["pix_per_cell"]
        cell_per_block = dist_pickle["cell_per_block"]
        spatial_size = dist_pickle["spatial_size"]
        hist_bins = dist_pickle["hist_bins"]
        y_start_stop = [None, None] 
        color_space = 'YCrCb'
        hog_channel = 'ALL'
        spatial_feat = True # Spatial features on or off
        hist_feat = True # Histogram features on or off
        hog_feat = True # HOG features on or off

        # Uncomment the following line if you extracted training
        # data from .png images (scaled 0 to 1 by mpimg) and the
        # image you are searching is a .jpg (scaled 0 to 255)
        image = image.astype(np.float32)/255

        windows = slide_window(image, x_start_stop=[None, None], y_start_stop=y_start_stop, 
                            xy_window=(96, 96), xy_overlap=(0.5, 0.5))

        hot_windows = search_windows(image, windows, svc, X_scaler, color_space=color_space, 
                                spatial_size=spatial_size, hist_bins=hist_bins, 
                                orient=orient, pix_per_cell=pix_per_cell, 
                                cell_per_block=cell_per_block, 
                                hog_channel=hog_channel, spatial_feat=spatial_feat, 
                                hist_feat=hist_feat, hog_feat=hog_feat)                       

        window_img = draw_boxes(draw_image, hot_windows, color=(0, 0, 255), thick=6)                    

        plt.imshow(window_img)
        plt.show()

    def visualize_identify_cars(self):
        image_file = './test_images/test6.jpg'
        image = mpimg.imread(image_file)

        colorspaces = ['RGB', 'HLS', 'HSV', 'YCrCb']

        for color_space in colorspaces:

            dist_pickle = self.pipeline.load_pickle("svc_pickle_" + color_space + ".p")
            svc = dist_pickle["svc"]
            X_scaler = dist_pickle["scaler"]
            orient = dist_pickle["orient"]
            pix_per_cell = dist_pickle["pix_per_cell"]
            cell_per_block = dist_pickle["cell_per_block"]
            spatial_size = dist_pickle["spatial_size"]
            hist_bins = dist_pickle["hist_bins"]
            test_accuracy = dist_pickle["test_accuracy"]
            color_space = dist_pickle["color_space"]

            print('Using:',orient,'orientations',pix_per_cell, 'pixels per cell and', cell_per_block,'cells per block')

            print('color_space:', color_space, ' accuracy =', test_accuracy)
            ystart = 400
            ystop = 656
            scale = 1.5
            boxes = find_cars(image, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, color_space)
            draw_img = np.copy(image)

            for box in boxes:
                cv2.rectangle(draw_img,box[0],box[1],(0,0,255),6) 

                fig = plt.figure()
                #plt.subplot(121)
                plt.imshow(draw_img)
                plt.title('Detected Cars - ' + color_space)
                #fig.tight_layout()
                #plt.show()
                fig.savefig('./output_images/vis-detectcars-' + color_space + '.png')


    def visualize_on_test_images(self):
        test_image_files = glob.glob('./test_images/*.jpg')
        fig = plt.figure()

        x = 1
        for image_file in test_image_files:
            print(image_file, x)
            image = mpimg.imread(image_file)
            heat = np.zeros_like(image[:,:,0]).astype(np.float)	
            draw_image = np.copy(image)

            boxes = self.pipeline.detect_cars(image)

            for box in boxes:
                cv2.rectangle(draw_image, box[0],box[1],(0,0,255),6) 

            plt.subplot(3, 2, x)
            plt.imshow(draw_image)
            plt.title(image_file)
            x = x + 1

        fig.tight_layout()
        fig.savefig('./output_images/vis-test-images.png')

        
    def visualize_heatmap(self, heat, box_list, image):

        # Add heat to each box in box list
        heat = add_heat(heat, box_list)

        # Apply threshold to help remove false positives
        heat = apply_threshold(heat, 1)

        # Visualize the heatmap when displaying    
        heatmap = np.clip(heat, 0, 255)

        # Find final boxes from heatmap using label function
        labels = label(heatmap)
        draw_img = draw_labeled_bboxes(np.copy(image), labels)

        fig = plt.figure()
        plt.subplot(121)
        plt.imshow(draw_img)
        plt.title('Car Positions')
        plt.subplot(122)
        plt.imshow(heatmap, cmap='hot')
        plt.title('Heat Map')
        fig.tight_layout()
        plt.show()

    def visualize_heat(self):

        test_image_files = glob.glob('./test_images/*.jpg')

        for image_file in test_image_files:
            #image_file = './test_images/test6.jpg'
            #pipeline.visualize_search_windows(image_file)
            image = mpimg.imread(image_file)
            heat = np.zeros_like(image[:,:,0]).astype(np.float)	
            #draw_image = np.copy(image)

            bbox_list = self.pipeline.detect_cars(image)
            self.visualize_heatmap(heat, bbox_list, image)

In [ ]:
v = Visualize()

In [ ]:
image_file = './test_images/test6.jpg'
v.visualize_search_windows(image_file)

In [ ]:
v.visualize_input_images()

In [ ]:
v.visualize_hog()

In [ ]:
v.visualize_identify_cars()

In [ ]:
v.visualize_on_test_images()

In [ ]:
v.visualize_heat()